# Kaggle Learn-and-compete `Poker Rule Induction`

## Data Description

You are provided with 25,010 poker hands in train.csv and 1,000,000 in test.csv. Each hand consists of five cards with a given suit and rank, drawn from a standard deck of 52. Suits and ranks are represented as ordinal categories:

```
S1 “Suit of card #1”
Ordinal (1-4) representing {Hearts, Spades, Diamonds, Clubs}
C1 “Rank of card #1”
Numerical (1-13) representing (Ace, 2, 3, ... , Queen, King)

...

S5 “Suit of card #5”
C5 “Rank of card #5”
```

Each row in the training set has the accompanying class label for the poker hand it comprises. The hands are omitted from the test set and must be predicted by participants. Hands are classified into the following ordinal categories:


```
0: Nothing in hand; not a recognized poker hand 
1: One pair; one pair of equal ranks within five cards
2: Two pairs; two pairs of equal ranks within five cards
3: Three of a kind; three equal ranks within five cards
4: Straight; five cards, sequentially ranked with no gaps
5: Flush; five cards with the same suit
6: Full house; pair + different rank three of a kind
7: Four of a kind; four equal ranks within five cards
8: Straight flush; straight + flush
9: Royal flush; {Ace, King, Queen, Jack, Ten} + flush
```
Note that the Straight flush and Royal flush hands are not representative of
the true domain because they have been over-sampled. The straight flush
is 14.43 times more likely to occur in the training set, while the royal flush is 129.82 times more likely.

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
from functools import reduce

#### Read data

In [2]:
test_dataset = pd.read_csv('test.csv', delimiter=',')

In [3]:
test_dataset

,id,S1,C1,S2,C2,S3,C3,S4,C4,S5,C5
0,1,1,10,2,2,3,3,3,8,1,1
1,2,2,13,3,5,3,7,4,6,1,4
2,3,1,3,1,11,2,8,2,1,2,4
3,4,1,6,3,3,4,7,1,8,3,11
4,5,2,10,3,4,1,6,2,12,2,6
5,6,1,4,3,10,2,11,2,6,1,7
6,7,1,10,3,8,1,4,3,11,3,9
7,8,2,11,3,8,1,1,1,11,2,3
8,9,3,4,1,1,1,3,3,5,3,6
9,10,3,12,2,1,1,3,1,2,3,10


In [4]:
train_dataset = pd.read_csv('train.csv', delimiter=',')

In [5]:
data = train_dataset[['S1', 'C1','S2', 'C2','S3', 'C3','S4', 'C4','S5', 'C5']]
labels = train_dataset[['hand']]

In [6]:
data[0:10]

,S1,C1,S2,C2,S3,C3,S4,C4,S5,C5
0,4,9,2,1,2,2,4,7,2,8
1,1,4,3,6,1,12,3,11,2,7
2,1,11,4,1,3,7,4,11,2,1
3,2,9,2,4,3,6,1,9,4,9
4,1,8,2,4,2,11,2,2,2,1
5,2,5,1,5,2,13,2,3,3,13
6,3,10,4,6,1,4,2,13,4,5
7,4,10,3,1,2,13,4,2,4,7
8,3,2,4,10,3,3,4,4,1,9
9,2,7,3,8,4,8,2,13,2,12


In [7]:
labels_vect = tf.keras.utils.to_categorical(labels)

#### One hot encoding

In [8]:
def data_hotencode(data, columns, encoder, mode='transform'):
    encoded = None
    if (mode == 'fit'):
        encoded = encoder.fit_transform(data[columns])
    elif (mode == 'transform'):
        encoded = encoder.transform(data[columns])
    return pd.concat([data.drop(columns, axis=1), pd.DataFrame(encoded.toarray())], axis=1)

def hand_hotencode(hand, columns, encoder):
    df = pd.DataFrame(data=[hand], columns=['S1', 'C1','S2', 'C2','S3', 'C3','S4', 'C4','S5', 'C5'])
    return data_hotencode(df, columns, encoder, 'transform')

In [9]:
def shuffle_columns(data, columns):
    shuffled = data[columns].reindex(np.random.permutation(data[columns].columns), axis=1)
    return pd.concat([data.drop(columns, axis=1), shuffled], axis=1)

In [9]:
encoder = OneHotEncoder()
columns_to_hotencode = ['S1', 'S2', 'S3', 'S4', 'S5']
data_encoded = data_hotencode(data, columns_to_hotencode, encoder, 'fit')
print(data_encoded.shape)

data_encoded.head()

(25010, 25)


,C1,C2,C3,C4,C5,0,1,2,3,4,...,10,11,12,13,14,15,16,17,18,19
0,9,1,2,7,8,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,4,6,12,11,7,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,11,1,7,11,1,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
3,9,4,6,9,9,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,8,4,11,2,1,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


#### Sort

In [79]:
def sort(data, columns, ace_is_max = False):
    sorted = data[columns]
    if ace_is_max:
        sorted = sorted.replace(1, 14) - 1
    else:
        sorted = sorted
    
    sorted.values.sort()
    return pd.concat([sorted, data.drop(columns, axis=1)], axis=1)

In [80]:
columns_to_sort = ['C1', 'C2', 'C3', 'C4', 'C5']
data_sorted = sort(data_encoded, columns_to_sort)
data_sorted.head()

,C1,C2,C3,C4,C5,0,1,2,3,4,...,10,11,12,13,14,15,16,17,18,19
0,1,2,7,8,9,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,4,6,7,11,12,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,1,1,7,11,11,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
3,4,6,9,9,9,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,1,2,4,8,11,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


#### Normalization

In [27]:
def normalize(data, columns, mean, std):
    data_normalized = (data[columns] - mean) / std
    to_return = pd.concat([data.drop(columns, axis=1), data_normalized], axis=1)
    to_return.columns = list(range(to_return.shape[1]))
    return to_return

In [28]:
columns_to_normalize = ['C1', 'C2', 'C3', 'C4', 'C5']
mean = data_sorted[columns_to_sort].mean().mean()
std = pd.Series(data_sorted[columns_to_sort].values.flatten()).std()
data_normalized = normalize(data_sorted, columns_to_normalize, mean, std)
data_normalized.head()

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,...,1.0,0.0,1.0,0.0,0.0,-1.596136,-1.329480,0.003798,0.270453,0.537109
1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,-0.796169,-0.262858,0.003798,1.070420,1.337076
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,-1.596136,-1.596136,0.003798,1.070420,1.070420
3,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,-0.796169,-0.262858,0.537109,0.537109,0.537109
4,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,-1.596136,-1.329480,-0.796169,0.270453,1.070420


#### 52 encoding

In [109]:
def data_52encode(data):
    columns = []
    for i in range(1, 5):
        for j in range(1, 14):
            columns.append(str(i) + str(j))
    
    data_squashed = pd.DataFrame()
    for i in range(1, 6):
        column_S = 'S'+str(i)
        column_C = 'C' + str(i)
        data_squashed[column_S+column_C] = data[column_S].astype(str) + data[column_C].astype(str)
    
    data_encoded = pd.DataFrame(data = np.zeros((data.shape[0], len(columns))), columns=columns)
    
    for i in range(data.shape[0]):
        row = data_squashed.iloc[i, :]
        for col in data_squashed.columns:
            data_encoded[row[col]][i] = 1
    
    return data_encoded

In [110]:
data_52encoded = data_52encode(data)

KeyboardInterrupt: 

In [111]:
def get_hand_df(hand):
    return pd.DataFrame({
        'S1': [hand[0]],
        'C1': [hand[1]],
        'S2': [hand[2]],
        'C2': [hand[3]],
        'S3': [hand[4]],
        'C3': [hand[5]],
        'S4': [hand[6]],
        'C4': [hand[7]],
        'S5': [hand[8]],
        'C5': [hand[9]],
    })

#### Test function

In [29]:
def calc_hand_label(hand):    
    def f(hand):
        suits_hist = reduce(lambda d,x: {**d, **{x: (d.get(x, 0) + 1)}}, hand[:,0], {})
        ranks_hist = reduce(lambda d,x: {**d, **{x: (d.get(x, 0) + 1)}}, hand[:,1], {})
        
        if len(ranks_hist.values()) < 5:
            if len(ranks_hist.values()) == 2:
                if max(list(ranks_hist.values())) == 4:
                    return 7
    #                 print('7: Four of a kind; four equal ranks within five cards')
                else:
                    return 6
    #                 print('6 Full house; pair + different rank three of a kind')
            elif len(ranks_hist.values()) == 3:
                if max(list(ranks_hist.values())) == 3:
                    return 3
    #                 print('3: Three of a kind; three equal ranks within five cards')
                else:
                    return 2
    #                 print('2: Two pairs; two pairs of equal ranks within five cards')
            else:
                return 1
    #             print('1: One pair; one pair of equal ranks within five cards')
        else:
            if len(suits_hist.values()) == 1:
                if max(list(ranks_hist.keys())) -  min(list(ranks_hist.keys())) == 4:
                    if max(list(ranks_hist.keys())) == 13:
                        return 9
    #                     print('9: Royal flush; {Ace, King, Queen, Jack, Ten} + flush')
                    else:
                        return 8
    #                     print('8: Straight flush; straight + flush')
                else:
                    return 5
    #                 print('5: Flush; five cards with the same suit')
            elif max(list(ranks_hist.keys())) -  min(list(ranks_hist.keys())) == 4:
                return 4    
    #             print('4: Straight; five cards, sequentially ranked with no gaps')
            else:
                return 0
                print('0: Nothing in hand; not a recognized poker hand ')
                
    hand = np.array(hand).reshape(5,2)
    res1 = f(hand)
    
    hand[:,1] = list(map(lambda x: 13 if (x == 1) else x-1, hand[:,1]))
    res2 = f(hand)
    
    if res1 == 9 and res2 == 8:
        return res2
    else:
        return max(res1, res2)

#### Encode cards

In [30]:
def card_encode(s, c):
    cards=["A","2","3","4","5","6","7","8","9","10","J","Q","K"]
    suits="♥♦♣♠"
    return cards[c - 1] + suits[s - 1]

def hand_encode(hand):
    return ', '.join(map(lambda x: card_encode(hand[x*2], hand[x*2+1]),range(5)))

print(hand_encode([4,9,2,1,2,2,4,7,2,8]))
print(hand_encode([2,9,2,4,3,6,1,9,4,9]))

9♠, A♦, 2♦, 7♠, 8♦
9♦, 4♦, 6♣, 9♥, 9♠


### Solutions

1. Neural network
2. Random forest

Ousiders:
 - Genetic algorithm
 - Decision tree

### Neural Network

In [31]:
def fit(model, data, labels, columns_to_shuffle, iterations, epochs, batch_size):
    for i in range(iterations):
        print ('-----------------------Iteration:' + str(i+1) +  '-----------------------')
        model.fit(np.array(data), np.array(labels), epochs=epochs, batch_size=batch_size)
        data = shuffle_columns(data, columns_to_shuffle)
    return model

In [53]:
from time import time
from keras.callbacks import TensorBoard

model = keras.Sequential()
#input
model.add(keras.layers.Dense(64, activation='relu', input_shape=(25,)))
model.add(keras.layers.Dropout(0.1))
#hidden
model.add(keras.layers.Dense(32, activation='relu'))
model.add(keras.layers.Dropout(0.05))
model.add(keras.layers.Dense(16, activation='relu'))
# Add a softmax layer with 10 output units:
model.add(keras.layers.Dense(10, activation='softmax'))

model.compile(optimizer=tf.train.AdamOptimizer(0.0005),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

tensorboard = TensorBoard(log_dir="logs/{}".format(time()))

model.fit(np.array(data_normalized), np.array(labels_vect), epochs=1500, batch_size=512)

Epoch 1/1500
25010/25010 [==============================]25010/25010 [==============================] - 0s 17us/step - loss: 2.1304 - acc: 0.2933

Epoch 2/1500
25010/25010 [==============================]25010/25010 [==============================] - 0s 8us/step - loss: 1.3871 - acc: 0.4995

Epoch 3/1500
25010/25010 [==============================]25010/25010 [==============================] - 0s 8us/step - loss: 1.0694 - acc: 0.4916

Epoch 4/1500
25010/25010 [==============================]25010/25010 [==============================] - 0s 8us/step - loss: 1.0192 - acc: 0.5127

Epoch 5/1500
25010/25010 [==============================]25010/25010 [==============================] - 0s 8us/step - loss: 0.9865 - acc: 0.5301

Epoch 6/1500
25010/25010 [==============================]25010/25010 [==============================] - 0s 8us/step - loss: 0.9612 - acc: 0.5492

Epoch 7/1500
25010/25010 [==============================]25010/25010 [==============================] - 0s 8us/step - loss:

25010/25010 [==============================]25010/25010 [==============================] - 0s 9us/step - loss: 0.1038 - acc: 0.9783

Epoch 57/1500
25010/25010 [==============================]25010/25010 [==============================] - 0s 8us/step - loss: 0.1059 - acc: 0.9773

Epoch 58/1500
25010/25010 [==============================]25010/25010 [==============================] - 0s 8us/step - loss: 0.0998 - acc: 0.9789

Epoch 59/1500
25010/25010 [==============================]25010/25010 [==============================] - 0s 8us/step - loss: 0.0934 - acc: 0.9811

Epoch 60/1500
25010/25010 [==============================]25010/25010 [==============================] - 0s 8us/step - loss: 0.0927 - acc: 0.9810

Epoch 61/1500
25010/25010 [==============================]25010/25010 [==============================] - 0s 8us/step - loss: 0.0909 - acc: 0.9814

Epoch 62/1500
25010/25010 [==============================]25010/25010 [==============================] - 0s 8us/step - loss: 0.0879 

25010/25010 [==============================]25010/25010 [==============================] - 0s 9us/step - loss: 0.0381 - acc: 0.9916

Epoch 112/1500
25010/25010 [==============================]25010/25010 [==============================] - 0s 8us/step - loss: 0.0395 - acc: 0.9918

Epoch 113/1500
25010/25010 [==============================]25010/25010 [==============================] - 0s 8us/step - loss: 0.0394 - acc: 0.9914

Epoch 114/1500
25010/25010 [==============================]25010/25010 [==============================] - 0s 8us/step - loss: 0.0382 - acc: 0.9916

Epoch 115/1500
25010/25010 [==============================]25010/25010 [==============================] - 0s 8us/step - loss: 0.0381 - acc: 0.9922

Epoch 116/1500
25010/25010 [==============================]25010/25010 [==============================] - 0s 8us/step - loss: 0.0361 - acc: 0.9920

Epoch 117/1500
25010/25010 [==============================]25010/25010 [==============================] - 0s 8us/step - loss: 0

25010/25010 [==============================]25010/25010 [==============================] - 0s 9us/step - loss: 0.0238 - acc: 0.9948

Epoch 167/1500
25010/25010 [==============================]25010/25010 [==============================] - 0s 8us/step - loss: 0.0228 - acc: 0.9946

Epoch 168/1500
25010/25010 [==============================]25010/25010 [==============================] - 0s 8us/step - loss: 0.0226 - acc: 0.9946

Epoch 169/1500
25010/25010 [==============================]25010/25010 [==============================] - 0s 8us/step - loss: 0.0213 - acc: 0.9951

Epoch 170/1500
25010/25010 [==============================]25010/25010 [==============================] - 0s 8us/step - loss: 0.0197 - acc: 0.9956

Epoch 171/1500
25010/25010 [==============================]25010/25010 [==============================] - 0s 8us/step - loss: 0.0220 - acc: 0.9953

Epoch 172/1500
25010/25010 [==============================]25010/25010 [==============================] - 0s 8us/step - loss: 0

25010/25010 [==============================]25010/25010 [==============================] - 0s 9us/step - loss: 0.0115 - acc: 0.9969

Epoch 222/1500
25010/25010 [==============================]25010/25010 [==============================] - 0s 8us/step - loss: 0.0117 - acc: 0.9967

Epoch 223/1500
25010/25010 [==============================]25010/25010 [==============================] - 0s 8us/step - loss: 0.0114 - acc: 0.9970

Epoch 224/1500
25010/25010 [==============================]25010/25010 [==============================] - 0s 8us/step - loss: 0.0127 - acc: 0.9969

Epoch 225/1500
25010/25010 [==============================]25010/25010 [==============================] - 0s 8us/step - loss: 0.0127 - acc: 0.9967

Epoch 226/1500
25010/25010 [==============================]25010/25010 [==============================] - 0s 8us/step - loss: 0.0113 - acc: 0.9966

Epoch 227/1500
25010/25010 [==============================]25010/25010 [==============================] - 0s 8us/step - loss: 0

...


Epoch 1257/1500
25010/25010 [==============================]25010/25010 [==============================] - 0s 9us/step - loss: 0.0011 - acc: 0.9999

Epoch 1258/1500
25010/25010 [==============================]25010/25010 [==============================] - 0s 9us/step - loss: 1.3650e-04 - acc: 1.0000

Epoch 1259/1500
25010/25010 [==============================]25010/25010 [==============================] - 0s 8us/step - loss: 8.4631e-04 - acc: 0.9997

Epoch 1260/1500
25010/25010 [==============================]25010/25010 [==============================] - 0s 8us/step - loss: 9.2974e-04 - acc: 0.9997

Epoch 1261/1500
25010/25010 [==============================]25010/25010 [==============================] - 0s 9us/step - loss: 4.4693e-04 - acc: 0.9999

Epoch 1262/1500
25010/25010 [==============================]25010/25010 [==============================] - 0s 8us/step - loss: 7.7912e-04 - acc: 0.9997

Epoch 1263/1500
25010/25010 [==============================]25010/25010 [============

25010/25010 [==============================]25010/25010 [==============================] - 0s 9us/step - loss: 3.2090e-04 - acc: 0.9998

Epoch 1311/1500
25010/25010 [==============================]25010/25010 [==============================] - 0s 8us/step - loss: 2.7860e-04 - acc: 0.9999

Epoch 1312/1500
25010/25010 [==============================]25010/25010 [==============================] - 0s 8us/step - loss: 4.9948e-04 - acc: 0.9999

Epoch 1313/1500
25010/25010 [==============================]25010/25010 [==============================] - 0s 8us/step - loss: 0.0012 - acc: 0.9999

Epoch 1314/1500
25010/25010 [==============================]25010/25010 [==============================] - 0s 8us/step - loss: 3.4277e-04 - acc: 1.0000

Epoch 1315/1500
25010/25010 [==============================]25010/25010 [==============================] - 0s 8us/step - loss: 8.9329e-04 - acc: 0.9999

Epoch 1316/1500
25010/25010 [==============================]25010/25010 [=============================

25010/25010 [==============================]25010/25010 [==============================] - 0s 9us/step - loss: 3.4187e-04 - acc: 0.9998

Epoch 1364/1500
25010/25010 [==============================]25010/25010 [==============================] - 0s 8us/step - loss: 5.9791e-04 - acc: 0.9998

Epoch 1365/1500
25010/25010 [==============================]25010/25010 [==============================] - 0s 8us/step - loss: 6.4393e-04 - acc: 0.9998

Epoch 1366/1500
25010/25010 [==============================]25010/25010 [==============================] - 0s 8us/step - loss: 7.0701e-04 - acc: 1.0000

Epoch 1367/1500
25010/25010 [==============================]25010/25010 [==============================] - 0s 8us/step - loss: 0.0011 - acc: 0.9998

Epoch 1368/1500
25010/25010 [==============================]25010/25010 [==============================] - 0s 8us/step - loss: 0.0010 - acc: 0.9997

Epoch 1369/1500
25010/25010 [==============================]25010/25010 [==============================] -

25010/25010 [==============================]25010/25010 [==============================] - 0s 8us/step - loss: 3.7404e-04 - acc: 1.0000

Epoch 1417/1500
25010/25010 [==============================]25010/25010 [==============================] - 0s 8us/step - loss: 9.5728e-04 - acc: 0.9998

Epoch 1418/1500
25010/25010 [==============================]25010/25010 [==============================] - 0s 8us/step - loss: 0.0012 - acc: 0.9998

Epoch 1419/1500
25010/25010 [==============================]25010/25010 [==============================] - 0s 8us/step - loss: 8.7092e-04 - acc: 0.9998

Epoch 1420/1500
25010/25010 [==============================]25010/25010 [==============================] - 0s 8us/step - loss: 6.2301e-04 - acc: 0.9999

Epoch 1421/1500
25010/25010 [==============================]25010/25010 [==============================] - 0s 8us/step - loss: 5.4286e-04 - acc: 0.9998

Epoch 1422/1500
25010/25010 [==============================]25010/25010 [=============================

25010/25010 [==============================]25010/25010 [==============================] - 0s 10us/step - loss: 4.2538e-04 - acc: 0.9999

Epoch 1470/1500
25010/25010 [==============================]25010/25010 [==============================] - 0s 11us/step - loss: 3.1054e-04 - acc: 1.0000

Epoch 1471/1500
25010/25010 [==============================]25010/25010 [==============================] - 0s 11us/step - loss: 5.0532e-04 - acc: 0.9999

Epoch 1472/1500
25010/25010 [==============================]25010/25010 [==============================] - 0s 10us/step - loss: 5.2753e-04 - acc: 0.9999

Epoch 1473/1500
25010/25010 [==============================]25010/25010 [==============================] - 0s 9us/step - loss: 2.4510e-04 - acc: 1.0000

Epoch 1474/1500
25010/25010 [==============================]25010/25010 [==============================] - 0s 9us/step - loss: 2.2771e-04 - acc: 1.0000

Epoch 1475/1500
25010/25010 [==============================]25010/25010 [=====================

In [33]:
y_test = test_dataset.drop(['id'], axis=1).apply(calc_hand_label, axis=1)

In [39]:
model.save('./model-sort-166')

### Predictions

In [34]:
test_dataset_noid = test_dataset.drop(['id'], axis=1)

In [66]:
test_sample = test_dataset_noid
test_data_sample_encoded_1 = normalize(
    sort(
        data_hotencode(test_sample, columns_to_hotencode, encoder),
        columns_to_sort,
        False,
    ), 
    columns_to_normalize,
    mean,
    std
)

test_data_sample_encoded_2 = normalize(
    sort(
        data_hotencode(test_sample, columns_to_hotencode, encoder),
        columns_to_sort,
        True,
    ), 
    columns_to_normalize,
    mean,
    std
)

In [56]:
model1 = keras.models.load_model('./model-sort-166')
model1.compile(optimizer=tf.train.AdamOptimizer(0.0005),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model2 = keras.models.load_model('./model-sort-174')
model2.compile(optimizer=tf.train.AdamOptimizer(0.0005),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [6]:
test_hand = [1,9,1,10,1,11,1,12,1,1]
print(hand_encode(test_hand))
prediction = model.predict(np.array(normalize(hand_hotencode(test_hand, columns_to_hotencode, encoder), columns_to_normalize, scaler)))
print(prediction[0])
plt.plot(prediction[0])

9♥, 10♥, J♥, Q♥, A♥


NameError: name 'model' is not defined

In [67]:
test_data_sample_predicted_1_10 = model1.predict(np.array(test_data_sample_encoded_1))
test_data_sample_predicted_1 = np.array(list(map(np.argmax, test_data_sample_predicted_1_10)))
test_data_sample_predicted_2_10 = model2.predict(np.array(test_data_sample_encoded_2))
test_data_sample_predicted_2 = np.array(list(map(np.argmax, test_data_sample_predicted_2_10)))

In [76]:
test_data_sample_predicted = np.maximum(test_data_sample_predicted_1, test_data_sample_predicted_2)
#test_data_sample_predicted = test_data_sample_predicted_2

test_data_sample_labels = y_test
df = pd.DataFrame({'predicted': test_data_sample_predicted, 'real': test_data_sample_labels})
df = pd.concat([test_sample, df], axis=1)
wrong_answers = df[df['predicted'] != df['real']]
print(wrong_answers.shape)
wrong_answers[0:10]

(117, 12)


,S1,C1,S2,C2,S3,C3,S4,C4,S5,C5,predicted,real
4943,2,2,4,1,2,5,4,3,4,4,8,4
5775,1,13,2,3,2,5,2,2,2,4,4,0
16549,2,13,2,2,1,10,1,12,2,11,4,0
16599,3,13,3,2,4,11,1,12,4,10,4,0
26585,4,12,4,11,2,2,4,13,2,10,4,0
42676,3,4,4,5,1,13,4,2,2,3,4,0
52860,1,13,1,5,2,3,4,4,4,2,4,0
53438,1,4,4,5,2,13,2,3,1,2,4,0
57315,3,10,3,13,3,12,1,2,2,11,4,0
60253,4,2,4,3,1,4,4,5,2,13,4,0


In [58]:
test_data_sample_predicted_1

array([0, 0, 0, ..., 1, 0, 1])

In [77]:
wrong_answers['real'].value_counts()

0    85
5     9
4     9
8     8
7     6
Name: real, dtype: int64

In [78]:
wrong_answers[0:10].drop(['predicted', 'real'], axis=1).apply(lambda h: hand_encode(np.array(h)), axis=1)

4943      2♦, A♠, 5♦, 3♠, 4♠
5775      K♥, 3♦, 5♦, 2♦, 4♦
16549    K♦, 2♦, 10♥, Q♥, J♦
16599    K♣, 2♣, J♠, Q♥, 10♠
26585    Q♠, J♠, 2♦, K♠, 10♦
42676     4♣, 5♠, K♥, 2♠, 3♦
52860     K♥, 5♥, 3♦, 4♠, 2♠
53438     4♥, 5♠, K♦, 3♦, 2♥
57315    10♣, K♣, Q♣, 2♥, J♦
60253     2♠, 3♠, 4♥, 5♠, K♦
dtype: object